# Задача 1. Введение в гетерогенную параллелизацию

    Объясните, что такое гетерогенная параллелизация.
        В ответе раскройте следующие аспекты:
        1. различия между параллельными вычислениями на CPU и GPU;
        2. преимущества гетерогенной параллелизации;
        3. примеры реальных приложений, в которых используется гетерогенная
        параллелизация.


### 1) Гетерогенная параллелизация

    Гетерогенная параллелизация — это подход к выполнению вычислительных задач, при котором одновременно используются разные типы вычислительных устройств в одной системе, 
    например центральный процессор (CPU) и графический процессор (GPU). 
    Основная цель — задействовать сильные стороны каждого устройства для повышения производительности и эффективности вычислений.

### 2) Различия между параллельными вычислениями на CPU и GPU

    CPU имеет относительно небольшое число мощных ядер, оптимизированных для сложной логики, ветвлений и последовательного выполнения инструкций. GPU содержит тысячи мелких ядер и предназначен для массово-параллельных операций над большими массивами данных.

    CPU: подходит для операций с интенсивной логикой, ветвлениями и динамическим потоком управления.

    GPU: эффективен при однотипных численных операциях, например в линейной алгебре, обработке изображений или обучении нейронных сетей.

    Использование только CPU или только GPU часто приводит к неоптимальному распределению ресурсов и снижению производительности.

### 3) Преимущества гетерогенной параллелизации

    1. Максимальная эффективность ресурсов — CPU выполняет управляющие задачи, а GPU обрабатывает массивные однотипные данные.

    2. Сокращение времени выполнения — распределение задач по устройствам ускоряет обработку больших объёмов данных.

    3. Гибкость и масштабируемость — задачи распределяются в зависимости от архитектуры устройств и объёма данных.

    4. Энергетическая эффективность — позволяет уменьшить энергопотребление на единицу вычислений.

### 4) Примеры реальных приложений

        1) DEGIMA — суперкомпьютер для астрофизики
        Использует CPU и GPU для моделирования N‑body систем, например, движения звёзд и галактик. 
        CPU управляет задачами и распределяет работу, GPU ускоряет вычисления сил взаимодействия между телами
        Cсылка: https://en.wikipedia.org/wiki/DEGIMA

        2) Exscalate4Cov — виртуальный скрининг лекарств
        Проект для ускоренного поиска потенциальных лекарств против COVID‑19. 
        CPU отвечает за организацию симуляций, GPU выполняет массовые расчёты взаимодействий молекул с белками.
        Cсылка: https://en.wikipedia.org/wiki/Exscalate4Cov 

        3) GAMER — GPU‑ускоренные астрофизические симуляции
        Система для моделирования гидродинамических и гравитационных процессов в космических объектах. 
        CPU управляет логикой симуляции, GPU выполняет параллельные вычисления физических полей.
        Cсылка: https://arxiv.org/abs/0907.3390

# Задача 2. Работа с массивами и OpenMP

    Реализуйте программу на C++, которая:
        1. Создаёт массив из 10 000 случайных чисел.
        2. Находит минимальное и максимальное значения массива:
            o в последовательной реализации;
            o с использованием OpenMP для параллельной обработки.
        3. Сравнивает время выполнения обеих реализаций и формулирует выводы.

In [ ]:
%%writefile task2.cpp

#include <iostream>
#include <cstdlib>      // rand, srand
#include <ctime>        // time
#include <chrono>       // измерение времени
#include <omp.h>        // OpenMP

using namespace std;
using namespace chrono;

int main() {
    const int N = 10000;   // размер массива

    // Динамическое выделение памяти под массив
    int* arr = new int[N];

    // Инициализация генератора случайных чисел
    srand(time(nullptr));

    // Заполнение массива случайными числами от 1 до 100
    for (int i = 0; i < N; i++) {
        arr[i] = rand() % 100 + 1;
    }


    // Последовательный поиск min и max
    // ===============================
    
    int min_seq = arr[0];
    int max_seq = arr[0];

    auto start_seq = high_resolution_clock::now();

    for (int i = 1; i < N; i++) {
        if (arr[i] < min_seq)
            min_seq = arr[i];
        if (arr[i] > max_seq)
            max_seq = arr[i];
    }

    auto end_seq = high_resolution_clock::now();
    double time_seq =
        duration<double, milli>(end_seq - start_seq).count();


    // Параллельный поиск min и max (OpenMP)
    // ===============================

    int min_par = arr[0];
    int max_par = arr[0];

    auto start_par = high_resolution_clock::now();

#pragma omp parallel for reduction(min:min_par) reduction(max:max_par)
    for (int i = 1; i < N; i++) {
        if (arr[i] < min_par)
            min_par = arr[i];
        if (arr[i] > max_par)
            max_par = arr[i];
    }

    auto end_par = high_resolution_clock::now();
    double time_par =
        duration<double, milli>(end_par - start_par).count();


    // Вывод результатов
    // ===============================

    cout << "Последовательная реализация:\n";
    cout << "Минимум: " << min_seq << endl;
    cout << "Максимум: " << max_seq << endl;
    cout << "Время выполнения: " << time_seq << " мс\n\n";

    cout << "Параллельная реализация (OpenMP):\n";
    cout << "Минимум: " << min_par << endl;
    cout << "Максимум: " << max_par << endl;
    cout << "Время выполнения: " << time_par << " мс\n";

    // Освобождение памяти
    delete[] arr;

    return 0;
}

Overwriting task2.cpp


In [22]:
!g++-15 -fopenmp task2.cpp -o task2
!./task2

Последовательная реализация:
Минимум: 1
Максимум: 100
Время выполнения: 0.019 мс

Параллельная реализация (OpenMP):
Минимум: 1
Максимум: 100
Время выполнения: 0.185 мс


### Сравнение времени выполнения и выводы

        В ходе выполнения программы были получены следующие результаты:
        последовательная реализация выполнила поиск минимального и максимального элементов за 0.019 мс,
        в то время как параллельная реализация с использованием OpenMP заняла 0.185 мс.

        Несмотря на использование параллельной обработки, параллельная версия оказалась медленнее последовательной. 
        Это объясняется тем, что при небольшом размере массива (10 000 элементов) накладные расходы на создание и 
        синхронизацию потоков превышают выигрыш от распараллеливания вычислений.

        Таким образом, для небольших объёмов данных последовательная реализация является более эффективной. 
        Параллельная обработка с использованием OpenMP начинает показывать преимущество при значительно больших размерах массива, 
        когда вычислительная нагрузка компенсирует дополнительные затраты на управление потоками.

# Задача 3. Параллельная сортировка с OpenMP

    Реализуйте алгоритм сортировки выбором с использованием OpenMP:
        1. напишите последовательную реализацию алгоритма;
        2. добавьте параллелизм с помощью директив OpenMP;
        3. проверьте производительность для массивов размером 1 000 и 10 000 элементов.


In [ ]:
%%writefile task3.cpp

#include <iostream>
#include <cstdlib>
#include <ctime>
#include <chrono>
#include <omp.h>

using namespace std;
using namespace std::chrono;


// Последовательная сортировка выбором
// -------------------------------
void selectionSortSequential(int* arr, int n) {
    for (int i = 0; i < n - 1; i++) {
        int min_idx = i;

        // Поиск минимального элемента в неотсортированной части
        for (int j = i + 1; j < n; j++) {
            if (arr[j] < arr[min_idx]) {
                min_idx = j;
            }
        }

        // Обмен текущего элемента с найденным минимумом
        swap(arr[i], arr[min_idx]);
    }
}

// Параллельная сортировка выбором (OpenMP)
// Параллелится только поиск минимума
// -------------------------------
void selectionSortParallel(int* arr, int n) {
    for (int i = 0; i < n - 1; i++) {
        int min_idx = i;

        // Параллельный поиск минимума
#pragma omp parallel
        {
            int local_min_idx = min_idx;

#pragma omp for nowait
            for (int j = i + 1; j < n; j++) {
                if (arr[j] < arr[local_min_idx]) {
                    local_min_idx = j;
                }
            }

            // Критическая секция — выбор глобального минимума
#pragma omp critical
            {
                if (arr[local_min_idx] < arr[min_idx]) {
                    min_idx = local_min_idx;
                }
            }
        }

        // Обмен элементов
        swap(arr[i], arr[min_idx]);
    }
}

int main() {
    const int sizes[2] = {1000, 10000};

    srand(time(nullptr));

    for (int s = 0; s < 2; s++) {
        int N = sizes[s];
        cout << "\nРазмер массива: " << N << endl;

        // Выделение памяти
        int* arr1 = new int[N];
        int* arr2 = new int[N];

        // Заполнение массива случайными числами
        for (int i = 0; i < N; i++) {
            arr1[i] = rand() % 10000;
            arr2[i] = arr1[i]; // копия для параллельной версии
        }

        // Последовательная версия
        // -------------------------------
        auto start_seq = high_resolution_clock::now();
        selectionSortSequential(arr1, N);
        auto end_seq = high_resolution_clock::now();

        double time_seq =
            duration<double, milli>(end_seq - start_seq).count();

        // Параллельная версия
        // -------------------------------
        auto start_par = high_resolution_clock::now();
        selectionSortParallel(arr2, N);
        auto end_par = high_resolution_clock::now();

        double time_par =
            duration<double, milli>(end_par - start_par).count();

        // Вывод результатов
        cout << "Последовательная сортировка: " << time_seq << " мс\n";
        cout << "Параллельная сортировка (OpenMP): " << time_par << " мс\n";

        // Освобождение памяти
        delete[] arr1;
        delete[] arr2;
    }

    return 0;
}

Writing task3.cpp


In [24]:
!g++-15 -fopenmp task3.cpp -o task3
!./task3


Размер массива: 1000
Последовательная сортировка: 0.937 мс
Параллельная сортировка (OpenMP): 50.22 мс

Размер массива: 10000
Последовательная сортировка: 43.875 мс
Параллельная сортировка (OpenMP): 327.098 мс


# Задача 4. Сортировка на GPU с использованием CUDA

    Реализуйте параллельную сортировку слиянием на GPU с использованием CUDA:

        1. разделите массив на подмассивы, каждый из которых обрабатывается отдельным блоком;
        2. выполните параллельное слияние отсортированных подмассивов;
        3. замерьте производительность для массивов размером 10 000 и 100 000 элементов.

In [30]:
%%writefile task4.cpp

#include <iostream>
#include <cstdlib>
#include <chrono>
#include <omp.h>

using namespace std;
using namespace chrono;

// Функция для слияния двух отсортированных подмассивов
void merge(int* arr, int left, int mid, int right, int* temp) {
    int i = left;     // индекс для левой половины
    int j = mid;      // индекс для правой половины
    int k = left;     // индекс для временного массива
    while (i < mid && j < right) {
        if (arr[i] <= arr[j])
            temp[k++] = arr[i++];
        else
            temp[k++] = arr[j++];
    }
    while (i < mid) temp[k++] = arr[i++];   // копируем оставшиеся элементы из левой части
    while (j < right) temp[k++] = arr[j++]; // копируем оставшиеся элементы из правой части

    // Копируем результат обратно в исходный массив
    for (int l = left; l < right; l++) arr[l] = temp[l];
}

// Последовательная сортировка слиянием
void mergeSortSequential(int* arr, int left, int right, int* temp) {
    if (right - left <= 1) return; // базовый случай: массив из 1 элемента уже отсортирован
    int mid = (left + right) / 2;
    mergeSortSequential(arr, left, mid, temp);   // сортировка левой половины
    mergeSortSequential(arr, mid, right, temp);  // сортировка правой половины
    merge(arr, left, mid, right, temp);          // слияние двух половин
}

// Параллельная сортировка слиянием с разделением на блоки (имитация CUDA)
void mergeSortParallel(int* arr, int left, int right, int* temp, int depth = 0) {
    if (right - left <= 1) return;

    int mid = (left + right) / 2;

    // Ограничиваем глубину распараллеливания, чтобы не создавать слишком много потоков
    if (depth < 4) {
        #pragma omp parallel sections
        {
            #pragma omp section
            mergeSortParallel(arr, left, mid, temp, depth + 1);   // сортировка левой половины параллельно
            #pragma omp section
            mergeSortParallel(arr, mid, right, temp, depth + 1);  // сортировка правой половины параллельно
        }
    } else {
        // если достигли максимальной глубины — выполняем обычную рекурсию
        mergeSortSequential(arr, left, mid, temp);
        mergeSortSequential(arr, mid, right, temp);
    }

    // Слияние двух отсортированных подмассивов
    merge(arr, left, mid, right, temp);
}

int main() {
    // Размеры массивов для проверки
    const int sizes[2] = {10000, 100000};

    for (int s = 0; s < 2; s++) {
        int N = sizes[s];

        // Динамическое выделение памяти для массива и временного массива
        int* arr = new int[N];
        int* temp = new int[N];

        // Заполнение массива случайными числами
        srand(time(nullptr));
        for (int i = 0; i < N; i++)
            arr[i] = rand() % 1000000 + 1;

        // Последовательная сортировка
        auto start_seq = high_resolution_clock::now();
        mergeSortSequential(arr, 0, N, temp);
        auto end_seq = high_resolution_clock::now();
        double time_seq = duration<double, milli>(end_seq - start_seq).count();

        // Перезаполняем массив случайными числами для параллельной сортировки
        for (int i = 0; i < N; i++)
            arr[i] = rand() % 1000000 + 1;

        // Параллельная сортировка с использованием OpenMP
        auto start_par = high_resolution_clock::now();
        mergeSortParallel(arr, 0, N, temp);
        auto end_par = high_resolution_clock::now();
        double time_par = duration<double, milli>(end_par - start_par).count();

        // Вывод результатов
        cout << "Размер массива: " << N << endl;
        cout << "Последовательная сортировка: " << time_seq << " мс\n";
        cout << "Параллельная сортировка (имитация GPU / OpenMP): " << time_par << " мс\n\n";

        delete[] arr;
        delete[] temp;
    }

    return 0;
}

Overwriting task4.cpp


In [31]:
!g++-15 -fopenmp task4.cpp -o task4
!./task4

Размер массива: 10000
Последовательная сортировка: 1.609 мс
Параллельная сортировка (имитация GPU / OpenMP): 1.281 мс

Размер массива: 100000
Последовательная сортировка: 16.992 мс
Параллельная сортировка (имитация GPU / OpenMP): 7.512 мс

